In [13]:
"""
@Description :   
@Author      :   Xubo Luo 
@Time        :   2024/04/22 19:22:54
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import warnings
import torch
warnings.filterwarnings("ignore")



In [49]:
path = './dataset/icml_face_data.csv'
icml_face_data = pd.read_csv(path)
train_data = pd.read_csv('./dataset/train.csv')
test_data = pd.read_csv('./dataset/test.csv')
fer2013 = pd.read_csv('./dataset/fer2013/fer2013.csv')

print(icml_face_data.info())
print(train_data.info())
print(test_data.info())
print(fer2013.info())


# visualize the data
# face0 = data[' pixels'][0]
# face0 = np.array(face0.split(' ')).astype(int)
# face0 = face0.reshape(48,48)
# plt.imshow(face0, cmap='gray')
# plt.show()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1    Usage   35887 non-null  object
 2    pixels  35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28709 entries, 0 to 28708
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  28709 non-null  int64 
 1   pixels   28709 non-null  object
dtypes: int64(1), object(1)
memory usage: 448.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7178 entries, 0 to 7177
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pixels  7178 non-null   object
dtypes: object(1)
memory usage: 56.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
D

In [5]:
width, height = 48, 48
datapoints = data[' pixels'].tolist()

X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

#getting labels for training
y = pd.get_dummies(data['emotion']).values

In [30]:
from sklearn.model_selection import train_test_split
import torch.utils.data as Data

num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train = X_train.transpose(0, 3, 1, 2)
X_train = np.repeat(X_train, 3, axis=1)
X_test = X_test.transpose(0, 3, 1, 2)
X_test = np.repeat(X_test, 3, axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)


train_dataset = Data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_dataset = Data.TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test ))

trainLoader = Data.DataLoader(dataset = train_dataset,batch_size = batch_size,num_workers = 0,shuffle = True)
testLoader = Data.DataLoader(dataset = test_dataset,batch_size = batch_size,num_workers = 0,shuffle = True)


In [40]:
from models.resnet.resnet_50 import ResNet50
import torch.nn.functional as F

model = ResNet50

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

for x_train, y_train in trainLoader:
    optimizer.zero_grad()
    output = model(x_train)
    
    # argmax to get the ground truth label
    y_train = torch.argmax(y_train, 1)


    print(output.shape)
    print(y_train.shape)

    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    print('Loss: {:.4f}'.format(loss.item()))

    break

torch.Size([64, 7])
torch.Size([64])
Loss: 2.0567


In [39]:
import torch
import torch.nn as nn

model = nn.Linear(10, 3)
criterion = nn.CrossEntropyLoss()

x = torch.randn(16, 10)
print(x)  # (16, 10)
y = torch.randint(0, 3, size=(16,))  # (16, )
print(y)  # (16, )
logits = model(x)  # (16, 3)
print(logits)  # (16, 3)

loss = criterion(logits, y)
print(loss.item())

tensor([[ 0.2979,  0.8008,  2.0456,  0.2136,  0.1391,  1.5940, -1.8350,  0.8807,
          1.7065,  0.0080],
        [ 0.6158, -0.3530,  1.5509,  0.3824, -1.2404, -1.3586, -1.0070,  0.8032,
          0.7509,  0.8942],
        [ 1.8139, -0.8261, -0.3482,  1.1907, -0.2112,  0.2066,  0.3866,  0.8093,
          0.7146,  0.0436],
        [ 0.4138,  0.6491,  0.5258, -0.6010,  1.1514, -1.2854,  2.1874,  0.5205,
          0.5926, -0.0681],
        [-0.4591,  0.6529, -0.8278,  0.8349, -0.1615, -0.5109, -0.0553, -0.1623,
          0.7694, -0.8767],
        [ 0.5022, -1.4322, -0.6836, -1.9829,  1.0481, -0.3302,  0.0765,  0.6034,
          0.7726, -0.7941],
        [-1.1848, -0.6320, -0.2534, -0.3789,  1.8789,  0.6126,  0.7054,  0.9618,
          0.6587, -0.1870],
        [ 0.5132,  0.9152, -0.3577,  0.3552,  0.6443,  0.4898,  0.6190,  0.6341,
         -0.8560, -0.0225],
        [-1.4122, -0.1423,  1.0570, -1.0622, -0.2023, -0.8812,  0.8438,  0.0076,
         -0.2508,  1.3412],
        [-0.4354, -

In [2]:
# filter the validation data from all data
import pandas as pd

data = pd.read_csv('./data/fer/icml_face_data.csv')
val_data = data[data['Usage'] == 'PublicTest']

print(val_data.info())

# save the validation data to a new csv file
val_data.to_csv('./data/fer/val.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 3589 entries, 28709 to 32297
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  3589 non-null   int64 
 1   Usage    3589 non-null   object
 2   pixels   3589 non-null   object
dtypes: int64(1), object(2)
memory usage: 112.2+ KB
None


In [2]:
import shutil
import os

train_input_path = './data/RAF-DB/DATASET/train'
train_output_path = './data/RAF-DB/DATASET/train_all'

if not os.path.exists(train_output_path):
    os.makedirs(train_output_path)

for root, dirs, files in os.walk(train_input_path):
    for file in files:
        if file.endswith('.jpg'):
            shutil.copy(os.path.join(root, file), train_output_path)

In [3]:
import shutil
import os

train_input_path = './data/RAF-DB/DATASET/test'
train_output_path = './data/RAF-DB/DATASET/test_all'

if not os.path.exists(train_output_path):
    os.makedirs(train_output_path)

for root, dirs, files in os.walk(train_input_path):
    for file in files:
        if file.endswith('.jpg'):
            shutil.copy(os.path.join(root, file), train_output_path)

In [5]:
import shutil
import os

train_input_path = './data/RAF-DB/DATASET/test'
train_output_path = './data/RAF-DB/DATASET/images'

if not os.path.exists(train_output_path):
    os.makedirs(train_output_path)

for root, dirs, files in os.walk(train_input_path):
    for file in files:
        if file.endswith('.jpg'):
            shutil.copy(os.path.join(root, file), train_output_path)